In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [42]:
# Angles (3) + Accel (3) + Compass(3)
#TODO 2 imus in so double data width?
#data_input_width = 3 + 3 + 3
data_input_width = 3 #use only angles for now
time_steps= 10

# Input Format
# [angles(t-time_steps+1), accel(t-time_steps+1), compass(t-time_steps+1)]
# ...
# [angles(t-2), accel(t-2), compass(t-2)]
# [angles(t-1), accel(t-1), compass(t-1)]
# [angles(t) accel(t), compass(t)]

#output Format
#
# [correct_exercise, overcompensation]
#

In [43]:
model = keras.Sequential()
#TODO I don't know if simpleRNN should come before or after LSTM
#TODO another input second IMU?
model.add(layers.SimpleRNN(32, input_shape = (time_steps, data_input_width), return_sequences=True))
model.add(layers.LSTM(16))
model.add(layers.Dense(10))
model.add(layers.Dense(2, activation='sigmoid')) #sigmoid has output between 0 and 1 (confidence)
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_19 (SimpleRNN)   (None, 10, 32)            1152      
                                                                 
 lstm_6 (LSTM)               (None, 16)                3136      
                                                                 
 dense_34 (Dense)            (None, 10)                170       
                                                                 
 dense_35 (Dense)            (None, 2)                 22        
                                                                 
Total params: 4,480
Trainable params: 4,480
Non-trainable params: 0
_________________________________________________________________


In [ ]:
N = 16    
Tp =12 

t=np.arange(0,N)
x=np.sin(0.02*t) #+2*np.random.rand(N)
df = pd.DataFrame(x)
df.head()

values=df.values
train,test = values[0:Tp,:], values[Tp:N,:]
step = 4
test = np.append(test,np.repeat(test[-1,],step))
train = np.append(train,np.repeat(train[-1,],step))

def convertToMatrix(data, step):
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
    return np.array(X), np.array(Y)

trainX,trainY =convertToMatrix(train,step)
testX,testY =convertToMatrix(test,step)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

print(trainX)
print(trainY)


In [63]:
def convertToMatrix(data, step):
    X = []
    for i in range(len(data) - step):
        d = i+step
        X.append(data[i:d,])
    return np.array(X)

def read_format_csv(filename):
    ignore_cols = ("time","IMU#","accelx","accely","accelz","compassx","compassy","compassz")
    data = pd.read_csv(filename)

    for col in ignore_cols:
        data.pop(col)

    return np.array(data)





inputs = convertToMatrix(read_format_csv("./data_simple/imu1_10_27.csv"), time_steps)
print(inputs[0])
print(inputs[1])
print(inputs)

outputs = None


[[-0.135  0.206 -1.473]
 [-0.114  0.214 -1.483]
 [-0.098  0.204 -1.535]
 [-0.103  0.176 -1.564]
 [-0.102  0.208 -1.519]
 [-0.124  0.15  -1.524]
 [-0.253  0.157 -1.339]
 [-0.521 -0.027 -1.221]
 [-0.827 -0.194 -1.298]
 [-0.8   -0.279 -1.474]]
[[-0.114  0.214 -1.483]
 [-0.098  0.204 -1.535]
 [-0.103  0.176 -1.564]
 [-0.102  0.208 -1.519]
 [-0.124  0.15  -1.524]
 [-0.253  0.157 -1.339]
 [-0.521 -0.027 -1.221]
 [-0.827 -0.194 -1.298]
 [-0.8   -0.279 -1.474]
 [-0.559 -0.282 -1.56 ]]
[[[-0.135  0.206 -1.473]
  [-0.114  0.214 -1.483]
  [-0.098  0.204 -1.535]
  ...
  [-0.521 -0.027 -1.221]
  [-0.827 -0.194 -1.298]
  [-0.8   -0.279 -1.474]]

 [[-0.114  0.214 -1.483]
  [-0.098  0.204 -1.535]
  [-0.103  0.176 -1.564]
  ...
  [-0.827 -0.194 -1.298]
  [-0.8   -0.279 -1.474]
  [-0.559 -0.282 -1.56 ]]

 [[-0.098  0.204 -1.535]
  [-0.103  0.176 -1.564]
  [-0.102  0.208 -1.519]
  ...
  [-0.8   -0.279 -1.474]
  [-0.559 -0.282 -1.56 ]
  [-0.464 -0.267 -1.677]]

 ...

 [[-6.392  4.02  -4.442]
  [-6.343  3.

In [ ]:
history = model.fit(
    inputs,
    outputs,
    epochs = 5,
    validation_split = 0.2,
)

model.save("./")


In [59]:
data_in = read_format_csv("./data_simple/imu0_10_27.csv")
prediction = model.predict(data_in)
print(prediction)
#TODO plot data_in and prediction

[[0.33891127 0.5295421 ]
 [0.33903322 0.5293829 ]
 [0.339416   0.52930474]
 ...
 [0.3814764  0.53624946]
 [0.38184732 0.53669876]
 [0.38250434 0.5369623 ]]
